# Measurement

In [ ]:
import time
from datetime import datetime

import numpy as np
from sciopy import (
    SystemMessageCallback_usb_hs,
    available_serial_ports,
    connect_COM_port,
    connect_COM_port_usb_hs,
    reshape_full_message_in_bursts,
    set_measurement_config_usb_hs,
    split_bursts_in_frames,
)
from sciopy.sciopy_dataclasses import ScioSpecMeasurementSetup

from src.classes import (
    BallAnomaly,
    Ender5Stat,
    HitBox,
    MeasurementInformation,
    TankProperties32x2,
)
from src.ender5 import (
    init_ender5,
    move_ender_to_coordinate,
    move_to_absolute_x_y_z,
    read_temperature,
    x_y_z_home,
)
from src.functions import (
    compute_hitbox,
    create_meas_coordinates,
    create_measurement_directory,
    create_mesh,
    empty_tank_measurement,
    print_coordinates_props,
    rename_savedir,
    save_parameters_to_json_file,
    set_perm,
)
from src.sciospec import sciospec_measurement
from src.visualization import plot_meas_coords, plot_meas_coords_wball, plot_mesh

## Define the parameters

In [ ]:
# init tank
tank = TankProperties32x2()
# init ball
ball = BallAnomaly(x=0, y=0, z=0, d=30, perm=10, material="acryl")
# init hitbox
hitbox = compute_hitbox(tank, ball, safety_tolerance=10.0)
# set min and max z-pos to ball.d/2
hitbox.z_min = tank.E_zr1 - ball.d /4
hitbox.z_max = tank.E_zr2 + ball.d /4
# set sciospec measurement properties
ssms = ScioSpecMeasurementSetup(
    burst_count=5,  # The number of measurements to be performed.
    total_meas_num=10,  # Repetitions of burst count
    n_el=64,  # Set 16, 32, 48 or 64 and do not forget to change the channel_group.
    channel_group=[
        1,
        2,
        3,
        4,
    ],  # Use [1] for n_el=16, [1,2] for n_el=32, [1,2,3] for n_el=48, and [1,2,3,4] for n_el=64
    exc_freq=100_000,  # 10,000Hz = 10kHz
    framerate=5,  # Measurements per second
    amplitude=0.01,  # 0.01A = 10mA (maximum)
    inj_skip=8,  # injection electrode skip
    gain=1,
    adc_range=1,  # +/- 1V
    notes="test measurement",  # add measurement information
    configured=True,
)

print(tank)
print(ball)
print(hitbox)
print(ssms)

In [ ]:
coordinates = create_meas_coordinates(hitbox, x_pts=15, y_pts=15, z_pts=15)
print_coordinates_props(coordinates)

In [ ]:
coordinates = np.round(coordinates,1)
coordinates

In [ ]:
print("total number of .npz files:", coordinates.shape[0] * ssms.burst_count)

In [ ]:
sel_coor_test = 0
plot_meas_coords(tank=tank, meas_coords=coordinates, p_select=sel_coor_test)

ball.x, ball.y, ball.z = coordinates[sel_coor_test]
print(ball)
# create an empty mesh
mesh_obj = create_mesh(tank)
set_perm(mesh_obj, anomaly=ball)
# plot mesh
plot_mesh(mesh_obj, tank, azim=15)

In [ ]:
s_path, f_name = create_measurement_directory()
save_parameters_to_json_file(s_path, f_name, ssms, tank, ball, hitbox)

## Connect Devices

- Create 3D objects
- Init Ender 5
- Centering object depending on size
- Init Sciospec
- Send config to sciospec
- Start Measurement

In [ ]:
COM_Ender = connect_COM_port(port="COM4", baudrate=115200)
COM_Sciospec = connect_COM_port_usb_hs()

In [ ]:
enderstat = Ender5Stat(
    abs_x_pos=None,
    abs_y_pos=None,
    abs_z_pos=None,
    tank_architecture=TankProperties32x2(),
    motion_speed=180,
)

In [ ]:
# init axis -> RUN ONLY IF NO OBJECT IS IN THE MOUNTING CONSTRUCTION
init_ender5(COM_Ender, enderstat)

## Set Sciospec Config

In [ ]:
set_measurement_config_usb_hs(COM_Sciospec, ssms)

In [ ]:
# Read out system callback
SystemMessageCallback_usb_hs(COM_Sciospec, prnt_msg=True)

## Measurement

In [ ]:
enderstat.abs_x_pos = 180
enderstat.abs_x_pos = 180
enderstat.abs_z_pos = 50
enderstat.motion_speed = 1500
move_to_absolute_x_y_z(COM_Ender, enderstat)

In [ ]:
print(
    f"In total this measurement will create {ssms.burst_count*coordinates.shape[0]} samples."
)

In [ ]:
ball.x, ball.y, ball.z = 180, 180, tank.T_bz[1] + ball.d
enderstat.abs_x_pos = 180
enderstat.abs_x_pos = 180
enderstat.abs_z_pos = tank.T_bz[1] + ball.d / 2
enderstat.motion_speed = 1500
move_to_absolute_x_y_z(COM_Ender, enderstat)

In [ ]:
# measurement information for documentation
today = datetime.now()
documentation = MeasurementInformation(
    saline=(9.75, "[ppt]"),
    saline_height=(131.0, "[mm]"),  # heigh without object
    temperature=(0.0, "[°C]"),
    timestamp=today.strftime("%d_%m_%Y_%Hh_%Mm"),
)

In [ ]:
# GT measurement befor start
empty_tank_measurement(
    COM_Ender,
    enderstat,
    COM_Sciospec,
    ssms,
    s_path,
    ball,
    documentation,
    sample_preamble="before",
    tank=tank,
)

In [ ]:
# move to start position
move_ender_to_coordinate(COM_Ender, coordinates[0], enderstat, print_msg=True)

In [ ]:
from tqdm import tqdm

In [ ]:
# start full measurement
samples_counter = 0
for XYZ in tqdm(coordinates):
    # update ball position
    ball.x, ball.y, ball.z = XYZ
    #mesh_obj = set_perm(mesh_obj, ball)
    #plot_mesh(mesh_obj, tank, show_tank_brdr=True)
    # move to position
    move_ender_to_coordinate(COM_Ender, XYZ, enderstat, print_msg=False)
    #time.sleep(4)
    # update documentation
    documentation.temperature = read_temperature(COM_Ender)
    current_time = datetime.now()
    documentation.timestamp = current_time.strftime("%d_%m_%Y_%Hh_%Mm")
    # measurement
    try:
        sciospec_data = sciospec_measurement(COM_Sciospec, ssms)
    except BaseException:
        #SystemMessageCallback(COM_ScioSpec)
        #time.sleep(5)
        #SoftwareReset(COM_ScioSpec)
        #time.sleep(10)
        #COM_ScioSpec = connect_COM_port(COM_ScioSpec.name)
        #time.sleep(2)
        #SystemMessageCallback(COM_ScioSpec)
        #scio_spec_measurement_config = conf_n_el_16_opposite(
        #    COM_ScioSpec, scio_spec_measurement_config
        #)
        time.sleep(1)
        SystemMessageCallback(COM_ScioSpec)
        time.sleep(4)
        SystemMessageCallback_usb_hs(COM_Sciospec, prnt_msg=True)
        time.sleep(1)
        set_measurement_config_usb_hs(COM_Sciospec, ssms)
        time.sleep(1)
        SystemMessageCallback_usb_hs(COM_Sciospec, prnt_msg=True)
        time.sleep(1)
        sciospec_data = sciospec_measurement(COM_Sciospec, ssms)
    save = True
    if save:
        for data in sciospec_data:
            # update documentation timestamp
            current_time = datetime.now()
            documentation.timestamp = current_time.strftime("%d_%m_%Y_%Hh_%Mm")

            np.savez(
                s_path + "sample_{0:06d}.npz".format(samples_counter),
                data=data,
                anomaly=ball,
                # mesh_obj=mesh_obj,
                config=ssms,
                tank=tank,
                documentation=documentation,
            )
            samples_counter += 1
        SystemMessageCallback_usb_hs(COM_Sciospec, prnt_msg=False)

In [ ]:
enderstat.abs_x_pos = 160
enderstat.abs_x_pos = 160
enderstat.abs_z_pos = 150
enderstat.motion_speed = 1500
move_to_absolute_x_y_z(COM_Ender, enderstat)

In [ ]:
SystemMessageCallback_usb_hs(COM_Sciospec)

In [ ]:
# measurement information for documentation
today = datetime.now()
documentation = MeasurementInformation(
    saline=(9.7, "[ppt]"),
    saline_height=documentation.saline_height,  # heigh without object
    temperature=(0.0, "[°C]"),
    timestamp=today.strftime("%d_%m_%Y_%Hh_%Mm"),
)
# GT measurement after measurement
empty_tank_measurement(
    COM_Ender,
    enderstat,
    COM_Sciospec,
    ssms,
    s_path,
    ball,
    documentation,
    sample_preamble="after",
    tank=tank,
)

In [ ]:
# rename the timestamp dir -> finishes the measurement
rename_savedir(s_path, ball=ball, ssms=ssms)
# x_y_z_home(COM_Ender, enderstat)